In this script we analyse the results from outlier detection using isolation forest.

The result csv was normalised but we matched it again with the original data.

In [1]:
import pandas as pd
from sqlalchemy import create_engine


dbname = 'DataMining'
user = 'postgres'
password = 'datamining'
host = 'localhost'  # localhost or the server address
port = '5433'  # default PostgreSQL port is 5432

# Establish a connection to the database
connection_str = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"
engine = create_engine(connection_str)

In [7]:
query = """
select count(*) from iso_forest_analysis;
"""

# Execute the query and fetch the data into a DataFrame
print(pd.read_sql_query(query, engine))

    count
0  176666


In [8]:
query = """
select count(*) from iso_forest_analysis
where (RS_E_RPM_PC1 = 0) and (RS_E_RPM_PC2 = 0);
"""

# Execute the query and fetch the data into a DataFrame
print(pd.read_sql_query(query, engine))

    count
0  133039


Here we see that a lot of the detected outliers are entries where the train is not moving. This is a result that could be expected. We can continue looking at the results where the train is moving (where one of the rpm values is not zero). We already analysed the points where the train is not moving (-> research question 12)

In [9]:
query = """
select * from iso_forest_analysis
where (RS_E_RPM_PC1 != 0) or (RS_E_RPM_PC2 != 0);
"""

# Execute the query and fetch the data into a DataFrame
df = pd.read_sql_query(query, engine)

stats = df[['rs_e_inairtemp_pc1', 'rs_e_inairtemp_pc2']].describe()

# Display the statistics
print(stats)


       rs_e_inairtemp_pc1  rs_e_inairtemp_pc2
count        43627.000000        43627.000000
mean           257.883078          373.117979
std           4006.746902         4853.955077
min              0.000000            0.000000
25%              3.000000            3.000000
50%              7.000000            7.000000
75%             14.500000           15.000000
max          65535.000000        65535.000000


So here we see the stats for rpm values that are not zero for both engines.

We can start by looking at the data for one vehicle id. Let's start with veh_id = 102

In [ ]:
df_102 = df[]